# **Import**

In [0]:
!pip install keras-rl
!pip install music21

In [0]:
from google.colab import files
import numpy as np
from music21 import stream, converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# **Data Import**

In [0]:
uploaded = files.upload()

fileNames = [];

for fn in uploaded.keys():
  fileNames.append(fn)

# **Data Processing**

In [0]:
'''
Parse all notes & chords in songs into strings
'''
notes = []

for file in fileNames:
  midi = converter.parse(file)
  print("Parsing {}".format(file))
  
  notes_to_parse = None
  try: 
    # file has instrument
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
      
print(notes)

In [0]:
'''
Create dictionary for notes
'''
pitchnames = sorted(set(item for item in notes))
dictionary = dict((note, number) for number,note in enumerate(pitchnames))

print(dictionary)

In [0]:
'''
Prepare training data
'''
WINDOW = 100

X = []
Y = []

for i in range(0, len(notes) - WINDOW, 1):
  x = notes[i:i + WINDOW]
  y = notes[i + WINDOW]
  X.append([dictionary[c] for c in x])
  Y.append(dictionary[y])
  
dataSetSize = len(X)

X = np.reshape(X, (dataSetSize, WINDOW, 1))
X = X / float(len(dictionary))

print(Y)
Y = np_utils.to_categorical(Y)

print(X.shape)
print(Y.shape)

# **Model Definition**

In [0]:
model = Sequential()

model.add(CuDNNLSTM(512, input_shape=(WINDOW, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(CuDNNLSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(dictionary)))
model.add(Activation('softmax'))

model.summary()

# **Model Training**

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

weights_filename = 'model_weights.h5f'
checkpoint = ModelCheckpoint(
    weights_filename,
    monitor='loss',
    verbose=0
)
callbacks = [checkpoint]

model.fit(
    X,
    Y,
    epochs=200,
    batch_size=200,
    callbacks=callbacks
)

# **Music Generation**

In [0]:
'''
Generate song
'''

songlength = 500

seed = np.random.randint(0, len(X) - 1)
reverse_dictionary = dict((number, note) for number, note in enumerate(pitchnames))

currentSequence = X[seed][:]
generatedSong = []

for i in range(songlength):
  x = np.reshape(currentSequence, (1, len(currentSequence,), 1))
  x = x / float(len(dictionary))
  
  p = model.predict(x, verbose=0)
  
  index = np.argmax(p)
  result = reverse_dictionary[index]
  generatedSong.append(result)
  
  currentSequence = np.append(currentSequence, index)
  currentSequence = currentSequence[1 : len(currentSequence)]

In [0]:
'''
Convert to midi
'''

offset = 0
output_notes = []

for sequence in generatedSong:
  #if sequence is chord
  if('.' in sequence) or sequence.isdigit():
    notes_in_chord = sequence.split('.')
    notes = []
    for n in notes_in_chord:
      new_n = note.Note(int(n))
      new_n.storedInstrument = instrument.Piano()
      notes.append(new_n)
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  #if sequence is note
  else:
    new_n = note.Note(sequence)
    new_n.offset = offset
    new_n.storedInstrument = instrument.Piano()
    output_notes.append(new_n)
    
  offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_ongaku.mid')